In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./freesound-audio-tagging/"))
import librosa
#from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import skew
SAMPLE_RATE = 44100

#from sklearn.model_selection import KFold, RepeatedKFold
from tqdm import tqdm, tqdm_pandas

tqdm.pandas()
import scipy
data_path = './freesound-audio-tagging/'
ss = pd.read_csv(os.path.join(data_path, 'sample_submission.csv'))

['train_feature_0.csv', 'test_feature_1.csv', 'audio_train.zip', 'audio_train', 'audio_test', 'sample_submission.csv', 'train_feature_1.csv', 'audio_test.zip', 'test_feature_0.csv', 'train.csv']


In [2]:
train_data = pd.read_csv(data_path + 'train_feature_1.csv')
test_data = pd.read_csv(data_path + 'test_feature_1.csv')
train_data.head()

,0,1,2,3,4,5,6,7,8,9,...,min_1_3,min_2_2,min_2_3,min_3_3,std_1_1,std_1_2,std_1_3,std_2_2,std_2_3,std_3_3
0,-446.838467,52.372300,-53.377823,-3.740264,-8.318184,-11.119042,-14.467405,-10.285843,-10.966904,-2.670278,...,-18601,-221,-615,-126,1206.881456,1706.545234,2088.675390,28.793127,83.189913,14.249761
1,-375.754678,151.572502,-41.933635,-0.165071,5.955898,-12.535086,-13.192531,-13.657687,-9.174413,-19.668802,...,-12593,-9359,-9359,-7484,3196.572491,3495.882224,3593.781554,2866.174942,3066.586209,2887.041732
2,-658.236035,102.368917,20.626247,9.108918,-4.891736,7.259463,-1.862530,1.661778,5.499974,5.338336,...,-636,-53,-132,-33,65.608624,91.744135,110.228966,13.855835,24.067880,13.421382
3,-770.905450,16.077204,-3.556132,-10.957762,-19.071291,-9.632298,4.279781,14.633798,19.459229,12.089039,...,-8345,-25,-83,-5,259.141991,366.449985,448.192027,4.846361,24.215311,1.041946
4,-343.603965,137.873119,-28.701815,8.593180,2.751699,-0.301433,-9.502459,-3.774746,13.056789,7.197487,...,-12829,-8860,-10921,-5577,3005.821294,3895.473242,3875.704209,1701.234020,3355.832221,905.514833


In [3]:
#loading data
train = pd.read_csv('./freesound-audio-tagging/train.csv')
submission = pd.read_csv('./freesound-audio-tagging/sample_submission.csv')

In [4]:

audio_train_files = os.listdir('./freesound-audio-tagging/audio_train')
audio_test_files = os.listdir('./freesound-audio-tagging/audio_test')

In [5]:
# Functions from LightGBM baseline: https://www.kaggle.com/opanichev/lightgbm-baseline
# Construct features set
X = train_data.drop(['label', 'fname'], axis=1)
feature_names = list(X.columns)
X = X.values
labels = np.sort(np.unique(train_data.label.values))
num_class = len(labels)
c2i = {}
i2c = {}
for i, c in enumerate(labels):
    c2i[c] = i
    i2c[i] = c
y = np.array([c2i[x] for x in train_data.label.values])
X = X[train['manually_verified'] == 1]
y = y[train['manually_verified'] == 1]
print(X.shape, y.shape)

((3710, 198), (3710,))


In [6]:
# fitting xgboost on the dataset
X_train, X_val, y_train, y_val= train_test_split(X, y, test_size=0.2, random_state=10, shuffle = True)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)
clf = XGBClassifier(max_depth=6, learning_rate=0.1, n_estimators=150,
                    n_jobs=-1, random_state=0, reg_alpha=0.2, 
                    colsample_bylevel=0.7, colsample_bytree=0.7)
clf.fit(X_train, y_train)
print(accuracy_score(clf.predict(X_val), y_val))
#more functions from LightGBM baseline: https://www.kaggle.com/opanichev/lightgbm-baseline
def proba2labels(preds, i2c, k=3):
    ans = []
    ids = []
    for p in preds:
        idx = np.argsort(p)[::-1]
        ids.append([i for i in idx[:k]])
        ans.append(' '.join([i2c[i] for i in idx[:k]]))

    return ans, ids

((2968, 198), (742, 198), (2968,), (742,))
0.7803234501347709


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [7]:
#fitting on the entire data

clf.fit(X, y)
str_preds, _ = proba2labels(clf.predict_proba(test_data.drop(['label', 'fname'], axis = 1).values), i2c, k=3)


In [9]:
# Prepare submission
subm = pd.DataFrame()
subm['fname'] = audio_test_files
subm['label'] = str_preds
#subm.to_csv('submission_03_180407.csv', index=False)